# Loading the database

In [1]:
import os
import json
import numpy as np
from mutagen.mp4 import MP4
from datetime import datetime
from bson import ObjectId
from pymongo import MongoClient
from pprint import PrettyPrinter
pp = PrettyPrinter()

In [2]:
MUSIC_DIR = "/Users/elizabethchen/Music/Library/"

In [3]:
IMAGE_DIR = "/Users/elizabethchen/Music/Images/"

In [4]:
def get_codec(mf_info_codec):
    if mf_info_codec == 'mp4a.40.2':
        return 'aac'
    elif mf_info_codec == 'alac':
        return 'alac'

In [5]:
for artist_dir in os.listdir(MUSIC_DIR):
    if not artist_dir.startswith("."):
        #print('Now showing contents of %s' % os.path.join(MUSIC_DIR, artist_dir))
        for album_dir in os.listdir(os.path.join(MUSIC_DIR, artist_dir)):
            if not album_dir.startswith("."):
                if not album_dir.endswith(".jpg"):
                    album_path = os.path.join(MUSIC_DIR, artist_dir, album_dir)
                    for song in os.listdir(album_path):
                        if not song.startswith("."):
                            if not song.endswith(".jpg"):
                                song_path = os.path.join(album_path, song)
                                mf = MP4(song_path)
                                title = mf['©nam'][0]
                                #title, artist, album, = mf['©nam'][0], mf['©ART'][0], mf['©alb'][0]
                                #album_artist, release_date = mf['aART'][0], mf['©day'][0]
                                track_num = mf['trkn'][0][0]
                                #print(title, track_num)
                                #duration, bitrate, codec = mf.info.length, mf.info.bitrate, mf.info.codec
                                

# MongoDB

In [6]:
connection = MongoClient("mongodb://127.0.0.1:27017/museplayer" )
db = connection.museplayer

### Artists Collection Structure

```
Artists: {
            _id: ObjectID
            artistName: string
            albums: [ list of album_ids ]
            profilePic: [image] or [filename/location]
         }
```

### Albums Collection Structure

```
Albums: { 
           _id: ObjectId
           albumName: string
           albumArtist: string
           releaseDate: date
           albumCover: filename/location
           genre: string
           songs: [ObjectId]
```

In [7]:
albums = db.albums
albums.drop()

artists = db.artists
artists.drop()

songs = db.songs
songs.drop()

for artist_name in os.listdir(MUSIC_DIR):
    if not artist_name.startswith("."):
        artist_album_ids = []
        for album_name in os.listdir(os.path.join(MUSIC_DIR, artist_name)):
            if not album_name in {'profile.jpg', '.DS_Store'}: # Two files to exclude when searching the artist directory
                cover_loc = os.path.join(IMAGE_DIR, artist_name, album_name, 'cover.jpg')
                album_id =  ObjectId()
                artist_id = ObjectId()
                songs_list = []
                for song in os.listdir(os.path.join(MUSIC_DIR, artist_name, album_name)):
                    if not song in {'cover.jpg', '.DS_Store'}: # Two files to exclude when searching the albums directory
                        try:
                            file_loc = os.path.join(MUSIC_DIR, artist_name, album_name, song)
                            mf = MP4(file_loc)
                            song_entry = {
                                '_id': ObjectId(),
                                'artistId': artist_id,
                                'albumId': album_id,
                                'songFileLoc': file_loc,
                                'albumArtworkLoc': cover_loc,
                                'title': mf['©nam'][0],
                                'duration': np.round(mf.info.length, 1),
                                'trackNumber': mf['trkn'][0][0],
                                'artistName': mf['©ART'][0],
                                'albumName': mf['©alb'][0],
                                'albumArtist': mf['aART'][0],
                                'releaseDate': mf['©day'][0],
                                'bitrate': np.round(mf.info.bitrate / 1000), 
                                'codec': get_codec(mf.info.codec),
                                'isFavorited': False,
                                'numPlays': 0, 
                                'lastPlayed': None
                            }
                            songs_list.append(song_entry["_id"])
                            songs.insert_one(song_entry)
                        except:
                            print(os.path.join(MUSIC_DIR, artist_name, album_name, song))
                artist_album_ids.append(album_id)
                album_entry  = {
                    '_id': album_id,
                    'artistId': artist_id,
                    'albumName': mf['©alb'][0],
                    'albumArtist': mf['aART'][0],
                    'releaseDate': mf['©day'][0],
                    'genre': mf['©gen'][0],
                    'songs': songs_list,
                    'albumArtworkLoc': cover_loc,
                    'isFavorited': False
                }
                albums.insert_one(album_entry)       
        profile_location = os.path.join(IMAGE_DIR, artist_name, 'profile.jpg')
        artist_entry = {
            '_id': artist_id,
            'artistName': mf['aART'][0],
            'artistAlbums': artist_album_ids,
            'profilePicLoc': profile_location,
            'isFavorited': False
        }
        artists.insert_one(artist_entry)

In [8]:
artists.create_index('artistName')
songs.create_index('title')
albums.create_index('albumName')

'albumName_1'